# libraries & global vars

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# io orgtree & org codes

In [2]:
path_org_str = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 5 - orgtree/"

In [3]:
filein_org_tree = "orgtree1_1.xlsx"

In [41]:
fileout_org_tree = "orgtree1_3"

# import newly parsed unique & multi strings

In [517]:
# read in orgtree db
orgtree_party = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="노동당")
orgtree_outside = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="당외곽")
orgtree_govt = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="정권기관")
orgtree_army = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="인민군")

In [518]:
orgtree_party.shape

(149, 10)

In [519]:
orgtree_govt.shape

(582, 10)

In [520]:
orgtree_army.shape

(289, 10)

In [521]:
orgtree_outside.shape

(391, 10)

In [522]:
orgtree_party.columns = orgtree_govt.columns

In [523]:
orgtree_army.columns = orgtree_govt.columns

In [524]:
orgtree_outside.columns = orgtree_govt.columns

In [525]:
orgtree_party.columns

Index(['ID', 'Org1', 'Org2', 'Org3', 'Org4', 'Org5', 'LinkToCurrent',
       'StatusUncertain', 'org1 error', '비고'],
      dtype='object')

In [526]:
orgtree_inside = pd.concat([orgtree_party,orgtree_army,orgtree_party,orgtree_govt])

In [527]:
orgtree_inside["InsideGov"] = True

In [528]:
orgtree_outside["InsideGov"] = False

In [541]:
orgtree = pd.concat([orgtree_inside,orgtree_outside])

In [542]:
orgtree.columns = ['ID', 'PrimaryInstitution', 'Org2', 'Org3', 'Org4', 'Org5', 'LinkToCurrent',
       'StatusUncertain', 'org1 error', '비고', 'InsideGov']

In [543]:
orgtree["OrgName"] = orgtree[["Org2","Org3","Org4","Org5"]].apply(lambda row: "_".join([str(item) for item in [row["Org2"],row["Org3"],row["Org4"],row["Org5"]] if item is not np.nan]),axis=1)

In [544]:
# remove all white spaces from PrimaryInstitution
orgtree["PrimaryInstitution"] = orgtree["PrimaryInstitution"].replace({" ":"","\n":""},regex=True)

In [545]:
# remove all white spaces from OrgName
orgtree["OrgName"] = orgtree["OrgName"].replace({" ":"","\n":""},regex=True)

In [551]:
# verify no PrimaryInstitution are ""
orgtree[orgtree.PrimaryInstitution == ""]

,ID,PrimaryInstitution,Org2,Org3,Org4,Org5,LinkToCurrent,StatusUncertain,org1 error,비고,InsideGov,OrgName


In [553]:
orgtree["ImmediateSuperiorOrg"] = orgtree["OrgName"].apply(lambda row: "_".join(row.split("_")[:-1]))

In [554]:
# replace "NAN" with proper null
orgtree.loc[orgtree.LinkToCurrent=="NAN"] = np.nan

In [555]:
orgtree[0:5]

,ID,PrimaryInstitution,Org2,Org3,Org4,Org5,LinkToCurrent,StatusUncertain,org1 error,비고,InsideGov,OrgName,ImmediateSuperiorOrg
0,1,당중앙군사위원회,NaN,NaN,NaN,NaN,Current,NaN,NaN,NaN,True,,
1,2,당중앙검사위원회,NaN,NaN,NaN,NaN,Current,NaN,NaN,NaN,True,,
2,3,당중앙위원회,정치국,NaN,NaN,NaN,Current,NaN,NaN,NaN,True,정치국,
3,4,당중앙위원회,비서국,NaN,NaN,NaN,Current,NaN,NaN,NaN,True,비서국,
4,5,당중앙위원회,조직지도부,NaN,NaN,NaN,Current,NaN,NaN,NaN,True,조직지도부,


In [556]:
orgtree.shape

(1560, 13)

In [557]:
orgtree = orgtree[~orgtree.ID.isna()]
orgtree.shape

(1470, 13)

In [558]:
select_columns = ["ID","InsideGov",
                  "PrimaryInstitution","OrgName","ImmediateSuperiorOrg",
                  "LinkToCurrent","StatusUncertain","org1 error","비고"]
orgtree_concise = orgtree[select_columns]

# remove StatusUncertain and org1 error

In [593]:
orgtree = orgtree_concise
orgtree.shape

(1470, 9)

In [594]:
# confirm no overlap between the two variables
orgtree[~orgtree["org1 error"].isna() & ~orgtree["StatusUncertain"].isna()]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent,StatusUncertain,org1 error,비고


In [595]:
orgtree_statusuncertain = orgtree[~orgtree["StatusUncertain"].isna()]
orgtree_statusuncertain.shape

(128, 9)

In [596]:
orgtree_org1error = orgtree[~orgtree["org1 error"].isna()]
orgtree_org1error.shape

(65, 9)

In [597]:
orgtree_no_errors = orgtree[orgtree["org1 error"].isna() & orgtree["StatusUncertain"].isna()]
orgtree_no_errors.shape

(1277, 9)

In [598]:
orgtree_no_errors.columns

Index(['ID', 'InsideGov', 'PrimaryInstitution', 'OrgName',
       'ImmediateSuperiorOrg', 'LinkToCurrent', 'StatusUncertain',
       'org1 error', '비고'],
      dtype='object')

In [599]:
# drop StatusUncertain & org1 error
orgtree_no_errors = orgtree_no_errors[['ID', 'InsideGov', 'PrimaryInstitution', 'OrgName','ImmediateSuperiorOrg', 'LinkToCurrent','비고']]
orgtree_no_errors.shape

(1277, 7)

In [600]:
# export rows with non-null StatusUncertain & org1 error
suffix = "_statusuncertain.xlsx"
orgtree_statusuncertain.to_excel(path_org_str+fileout_org_tree+suffix,index=False)

In [601]:
suffix = "_org1error.xlsx"
orgtree_statusuncertain.to_excel(path_org_str+fileout_org_tree+suffix,index=False)

# remove duplicate rows and IDs

In [603]:
orgtree = orgtree_no_errors.copy()
orgtree.shape

(1277, 7)

In [604]:
# remove duplicate rows
orgtree.drop_duplicates(keep="first",inplace=True,ignore_index=True)
orgtree.shape

(1172, 7)

In [605]:
orgtree[0:5]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent,비고
0,1,True,당중앙군사위원회,,,Current,NaN
1,2,True,당중앙검사위원회,,,Current,NaN
2,3,True,당중앙위원회,정치국,,Current,NaN
3,4,True,당중앙위원회,비서국,,Current,NaN
4,5,True,당중앙위원회,조직지도부,,Current,NaN


In [606]:
# find duplicate IDs
orgtree_duplicate_IDs = orgtree[orgtree.duplicated(subset="ID",keep=False)].sort_values("ID")
orgtree_duplicate_IDs.shape

(156, 7)

In [607]:
# replace duplicate IDs with null
orgtree.loc[orgtree.duplicated(subset="ID",keep=False),"ID"] = np.nan
orgtree.shape

(1172, 7)

In [608]:
# anymore duplicate rows?
orgtree[orgtree.duplicated()]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent,비고
316,NaN,True,조선인민군항공및반항공군,제8수송연대,,Current,NaN
319,NaN,True,조선인민군항공및반항공군,제8수송연대,,Current,NaN


In [609]:
# remove duplicate rows
orgtree.drop_duplicates(keep="first",inplace=True,ignore_index=True)
orgtree.shape

(1170, 7)

In [610]:
# export rows with duplicate IDs
suffix = "_duplicateIDs.xlsx"
orgtree_duplicate_IDs.to_excel(path_org_str+fileout_org_tree+suffix,index=False)

In [612]:
orgtree_cleanIDs = orgtree.copy()

# replace LinkToCurrent with PrimaryInstitution and OrgName

In [613]:
orgtree = orgtree_cleanIDs.copy()
orgtree.shape

(1170, 7)

In [614]:
# LinkToCurrent is Current
orgtree[orgtree.LinkToCurrent == "Current"].shape

(837, 7)

In [615]:
# LinkToCurrent is null
orgtree[orgtree.LinkToCurrent.isna()].shape

(38, 7)

In [616]:
# LinkToCurrent links to an ID
orgtree[(~orgtree.LinkToCurrent.isna()) & (orgtree.LinkToCurrent != "Current")].shape

(295, 7)

In [617]:
# create a unique list of non-null IDs
orgid = orgtree.loc[~orgtree.ID.isna(),["ID","PrimaryInstitution","OrgName"]]
orgid.shape

(1016, 3)

In [618]:
# verify no duplicated IDs
orgid[orgid.duplicated()]

,ID,PrimaryInstitution,OrgName


In [619]:
# verify no non-null IDs
orgid[orgid.ID.isna()]

,ID,PrimaryInstitution,OrgName


In [620]:
# merge orgtree with unique list of non-null IDs
orgtree_linked = orgtree.merge(orgid,how="left",left_on="LinkToCurrent",right_on="ID",suffixes=("","_LinkToCurrent"),indicator=True)
orgtree_linked.shape

(1170, 11)

In [621]:
# verify # of current is same
orgtree_linked[orgtree_linked.LinkToCurrent == "Current"].shape

(837, 11)

In [622]:
# verify # of null is same
orgtree_linked[orgtree_linked.LinkToCurrent.isna()].shape

(38, 11)

In [623]:
# verify # of non-null links is the same
orgtree_linked[(~orgtree_linked.LinkToCurrent.isna()) & (orgtree_linked.LinkToCurrent != "Current")].shape

(295, 11)

In [624]:
# all LinkToCurrent which failed to merge (excluding NaN and Current)
# some LinkToCurrent with non-null ID may not find a match, and will be null, if ID was a duplicate
orgtree_linked[((~orgtree_linked.LinkToCurrent.isna()) 
                & (orgtree_linked.LinkToCurrent != "Current")) 
               & (orgtree_linked["_merge"]=="left_only")].shape

(47, 11)

In [625]:
# code PrimaryInstitution_LinkToCurrent and OrgName_LinkToCurrent as Current if LinkToCurrent == LinkToCurrent
orgtree_linked.loc[orgtree_linked.LinkToCurrent=="Current","PrimaryInstitution_LinkToCurrent"] = "Current"
orgtree_linked.loc[orgtree_linked.LinkToCurrent=="Current","OrgName_LinkToCurrent"] = "Current"

In [626]:
# drop ID, LinkToCurrent, ID_LinkToCurrent, _merge
orgtree_linked.drop(columns=["ID","LinkToCurrent","ID_LinkToCurrent","_merge"],inplace=True)

In [627]:
# rename PrimaryInstitution_LinkToCurrent --> LinkToCurrent_PrimaryInstitution, LinkToCurrent_OrgName
orgtree_linked.columns = ['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg','비고',
       'LinkToCurrent_PrimaryInstitution', 'LinkToCurrent_OrgName']

In [628]:
# re-order columns
orgtree_linked = orgtree_linked[['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'LinkToCurrent_PrimaryInstitution', 'LinkToCurrent_OrgName','비고']]

In [629]:
orgtree_linked[0:5]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고
0,True,당중앙군사위원회,,,Current,Current,NaN
1,True,당중앙검사위원회,,,Current,Current,NaN
2,True,당중앙위원회,정치국,,Current,Current,NaN
3,True,당중앙위원회,비서국,,Current,Current,NaN
4,True,당중앙위원회,조직지도부,,Current,Current,NaN


In [630]:
orgtree_linked.shape

(1170, 7)

# remove duplicate PrimaryInstitution, OrgName
- verify every duplicated pair has at least one LinkToCurrent_PrimaryInstitution == Current
- keep the Current one and drop the other

In [632]:
orgtree = orgtree_linked.copy()
orgtree.shape

(1170, 7)

In [633]:
# make sure that InsideGov & ImmediateSuperiorOrg are unique

In [634]:
# verify InsideGov is unique and not duplicated
orgtree_counts = orgtree.groupby(["PrimaryInstitution","OrgName"])["InsideGov"].apply(lambda x:len(set(x))).reset_index()
orgtree_counts[orgtree_counts.InsideGov > 1]

,PrimaryInstitution,OrgName,InsideGov


In [635]:
# verify ImmediateSuperiorOrg is unique and not duplicated
orgtree_counts = orgtree.groupby(["PrimaryInstitution","OrgName"])["ImmediateSuperiorOrg"].apply(lambda x:len(set(x))).reset_index()
orgtree_counts[orgtree_counts.ImmediateSuperiorOrg > 1]

,PrimaryInstitution,OrgName,ImmediateSuperiorOrg


In [636]:
orgtree.shape

(1170, 7)

In [637]:
# verify every duplicated pair has at least one LinkToCurrent_PrimaryInstitution == Current
orgtree[orgtree[["PrimaryInstitution","OrgName"]].duplicated(keep=False)].sort_values(["PrimaryInstitution","OrgName","LinkToCurrent_PrimaryInstitution"])

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고
517,True,내각,,,Current,Current,NaN
668,True,내각,,,NaN,NaN,NaN
669,True,내각,,,NaN,NaN,NaN
519,True,내각,건설건재공업성,,Current,Current,NaN
602,True,내각,건설건재공업성,,내각,건설건재공업성,NaN
520,True,내각,경공업성,,Current,Current,NaN
614,True,내각,경공업성,,내각,경공업성,NaN
437,True,내각,교육위원회,,Current,Current,NaN
676,True,내각,교육위원회,,내각,교육위원회,NaN
451,True,내각,교육위원회_고등교육성,교육위원회,Current,Current,NaN


In [638]:
orgtree[orgtree[["PrimaryInstitution","OrgName"]].duplicated(keep=False)].sort_values(["PrimaryInstitution","OrgName","LinkToCurrent_PrimaryInstitution"]).shape

(89, 7)

In [639]:
# keep the Current one and drop the other
orgtree.sort_values(["PrimaryInstitution","OrgName","LinkToCurrent_PrimaryInstitution"],inplace=True)
orgtree.drop_duplicates(["PrimaryInstitution","OrgName"],keep="first",inplace=True,ignore_index=True)
orgtree.shape

(1125, 7)

In [640]:
# verify no rows with duplicate PrimaryInstitution & OrgName
orgtree[orgtree[["PrimaryInstitution","OrgName"]].duplicated(keep=False)]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고


In [642]:
orgtree_unique = orgtree.copy()

# recursively add ImmedSuperiorOrgs, drop duplicates and sort

In [643]:
orgtree = orgtree_unique.copy()
orgtree.shape

(1125, 7)

In [644]:
orgtree[0:2]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고
0,False,6.15공동선언실천남북공동위원회북측위원회,교직원분과,,Current,Current,NaN
1,False,6.15공동선언실천남북공동위원회북측위원회,노동자분과,,Current,Current,NaN


In [645]:
orgtree_2order = orgtree.loc[orgtree.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_2order["OrgName"] = orgtree_2order["OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_2order.drop_duplicates(inplace=True)
orgtree_2order.shape

(120, 3)

In [646]:
orgtree_3order = orgtree_2order.loc[orgtree_2order.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_3order["OrgName"] = orgtree_3order["OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_3order.drop_duplicates(inplace=True)
orgtree_3order.shape

(19, 3)

In [647]:
orgtree_4order = orgtree_3order.loc[orgtree_3order.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_4order.OrgName = orgtree_4order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_4order.drop_duplicates(inplace=True)
orgtree_4order.shape

(2, 3)

In [648]:
orgtree_5order = orgtree_4order.loc[orgtree_4order.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_5order.shape

(0, 3)

In [649]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'LinkToCurrent_PrimaryInstitution', 'LinkToCurrent_OrgName', '비고'],
      dtype='object')

In [650]:
orgtree_higher_order = pd.concat([orgtree_2order,orgtree_3order,orgtree_4order])
orgtree_higher_order.drop_duplicates(inplace=True)
orgtree_higher_order.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=False,inplace=True)
orgtree_higher_order.shape

(121, 3)

In [651]:
# find new orgtree_higher_order whose PrimaryOrg & OrgName are not contained in orgtree
orgtree_higher_order_merged = orgtree_higher_order.merge(orgtree[["PrimaryInstitution","OrgName"]],
                                             how="left",on=["PrimaryInstitution","OrgName"],
                                             suffixes=("","_right"),indicator=True)
orgtree_higher_order_unique = orgtree_higher_order_merged[orgtree_higher_order_merged["_merge"]=="left_only"]
orgtree_higher_order_unique.shape

(37, 4)

In [652]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'LinkToCurrent_PrimaryInstitution', 'LinkToCurrent_OrgName', '비고'],
      dtype='object')

In [653]:
orgtree_higher_order_unique.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', '_merge'], dtype='object')

In [654]:
# drop _merge column in orgtree_higher_order_unique
orgtree_higher_order_unique = orgtree_higher_order_unique.drop(columns=["_merge"],axis=1)
orgtree_higher_order_unique.shape

(37, 3)

In [655]:
# add ImmediateSuperiorOrg to orgtree_higher_order_unique
orgtree_higher_order_unique["ImmediateSuperiorOrg"] = orgtree_higher_order_unique["OrgName"].apply(lambda row: "_".join(row.split("_")[:-1]))

In [656]:
# add columns [LinkToCurrent_PrimaryInstitution, LinkToCurrent_OrgName, 비고] and set them to null
orgtree_higher_order_unique["LinkToCurrent_PrimaryInstitution"] = np.nan
orgtree_higher_order_unique["LinkToCurrent_OrgName"] = np.nan
orgtree_higher_order_unique["비고"] = np.nan

In [657]:
# concatenate orgtree and orgtree_higher_order_unique
orgtree_concat = pd.concat([orgtree,orgtree_higher_order_unique])
orgtree_concat.shape

(1162, 7)

In [658]:
# confirm no duplicate (PrimaryInstitution, OrgName) pairs 
orgtree_concat[orgtree_concat.duplicated(["PrimaryInstitution","OrgName"])]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고


In [659]:
# sort
orgtree_concat.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=False,inplace=True,ignore_index=True)

In [660]:
orgtree_concat.shape

(1162, 7)

In [661]:
orgtree_concat[0:5]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고
0,True,최고재판소,,,최고인민회의,중앙재판소,NaN
1,True,최고인민회의,황해북도인민위원회_서흥군인민위원회,황해북도인민위원회,NaN,NaN,NaN
2,True,최고인민회의,황해북도인민위원회_개성시인민위원회,황해북도인민위원회,NaN,NaN,NaN
3,True,최고인민회의,황해북도인민위원회,,NaN,NaN,NaN
4,True,최고인민회의,황해남도인민위원회,,NaN,NaN,NaN


# export orgtree_concise

In [662]:
# if LinkToCurrent_PrimaryInstitution, LinkToCurrent_OrgName are null, then indicate revision is necessary
orgtree_concat.loc[orgtree_concat.LinkToCurrent_PrimaryInstitution.isna(),"LinkToCurrent_PrimaryInstitution"] = "Please_Revise"
orgtree_concat.loc[orgtree_concat.LinkToCurrent_OrgName.isna(),"LinkToCurrent_OrgName"] = "Please_Revise"

In [663]:
orgtree_concat[0:5]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,비고
0,True,최고재판소,,,최고인민회의,중앙재판소,NaN
1,True,최고인민회의,황해북도인민위원회_서흥군인민위원회,황해북도인민위원회,Please_Revise,Please_Revise,NaN
2,True,최고인민회의,황해북도인민위원회_개성시인민위원회,황해북도인민위원회,Please_Revise,Please_Revise,NaN
3,True,최고인민회의,황해북도인민위원회,,Please_Revise,Please_Revise,NaN
4,True,최고인민회의,황해남도인민위원회,,Please_Revise,Please_Revise,NaN


In [664]:
orgtree_concat.to_excel(path_org_str+fileout_org_tree+".xlsx",index=False)